# Analysis of material properties of mitochondrial membranes

## Loading and setup

In [1]:
%load_ext autoreload

In [2]:
%matplotlib inline
%autoreload 1
import pickle
import numpy as np
from functools import partial
import MDAnalysis

from pathlib import Path

import matplotlib.pyplot as plt
import numpy.typing as npt

import pandas as pd

from scipy import integrate, interpolate, stats
from scipy.optimize import curve_fit
from tqdm.auto import tqdm
from tqdm.contrib.concurrent import process_map

%aimport util
from plot_helper import *

Matplotlib Version: 3.6.2


In [3]:
def radial_averaging(power2D, mc, min_bin=0.001, max_bin=1, bin_width=0.001):
    """
    Radially average the power spectrum to obtain values. Notably the natural freqeuncy unit
    of this function is A^-1.

    Args:
        power2D (numpy.array((N,N))): Power spectrum
        mc (_type_): Membrane curvature object with metadata
        min_bin (float, optional): Minimum bin value. Defaults to 0.001.
        max_bin (int, optional): Maximum bin value. Defaults to 1.
        bin_width (float, optional): Bin width. Defaults to 0.001.

    Returns:
        tuple: Binned power spectra
    """
    x, y = np.meshgrid(mc['qx'], mc['qy'])  # A^-1
    r = np.sqrt(x**2 + y**2)
    bins = np.arange(min_bin, max_bin, bin_width)

    digitized = np.digitize(r, bins)
    bc = np.array(
        [
            r[digitized == i].mean() if np.count_nonzero(digitized == i) else np.NAN
            for i in range(1, len(bins))
        ]
    )
    bm = np.array(
        [
            power2D[digitized == i].mean()
            if np.count_nonzero(digitized == i)
            else np.NAN
            for i in range(1, len(bins))
        ]
    )

    bin_centers = bc[np.isfinite(bm)]
    bin_means = bm[np.isfinite(bm)]

    return np.column_stack((bin_centers, bin_means, bin_centers**4 * bin_means))


def radial_averaging_series(power2D, mc, min_bin=0.001, max_bin=1, bin_width=0.001):
    """
    Radially average the power spectrum to obtain values. Notably the natural freqeuncy unit
    of this function is A^-1.

    Args:
        power2D (numpy.array((M,N,N))): Power spectrum
        mc (_type_): Membrane curvature object with metadata
        min_bin (float, optional): Minimum bin value. Defaults to 0.001.
        max_bin (int, optional): Maximum bin value. Defaults to 1.
        bin_width (float, optional): Bin width. Defaults to 0.001.

    Returns:
        tuple: Binned power spectra
    """

    if not len(power2D.shape) == 3:
        raise RuntimeError("Expected time series of 2D power")

    x, y = np.meshgrid(mc["qx"], mc["qy"])  # A^-1
    r = np.sqrt(x**2 + y**2)
    bins = np.arange(min_bin, max_bin, bin_width)

    digitized = np.digitize(r, bins)
    bc = np.array(
        [
            r[digitized == i].mean() if np.count_nonzero(digitized == i) else np.NAN
            for i in range(1, len(bins))
        ]
    )

    first_iter = True

    spectra = None

    for i, frame in tqdm(enumerate(power2D), total=len(power2D)):
        bm = np.array(
            [
                frame[digitized == i].mean()
                if np.count_nonzero(digitized == i)
                else np.NAN
                for i in range(1, len(bins))
            ]
        )

        if i == 0:
            bin_centers = bc[np.isfinite(bm)]
            bin_means = bm[np.isfinite(bm)]
            spectra = np.zeros((power2D.shape[0], len(bin_means)))
            spectra[i] = bin_means
        else:
            spectra[i] = bm[np.isfinite(bm)]
    return (bin_centers, spectra)


def radial_averaging_nm(power2D, mc, min_bin=0.1, max_bin=10, bin_width=0.1):
    x, y = np.meshgrid(mc.qx * 10, mc.qy * 10)  # convert to nm^-1
    r = np.sqrt(x**2 + y**2)
    bins = np.arange(min_bin, max_bin, bin_width)

    digitized = np.digitize(r, bins)
    bc = np.array(
        [
            r[digitized == i].mean() if np.count_nonzero(digitized == i) else np.NAN
            for i in range(1, len(bins))
        ]
    )
    bm = np.array(
        [
            power2D[digitized == i].mean()
            if np.count_nonzero(digitized == i)
            else np.NAN
            for i in range(1, len(bins))
        ]
    )

    bin_centers = bc[np.isfinite(bm)]
    bin_means = bm[np.isfinite(bm)]

    return np.column_stack((bin_centers, bin_means, bin_centers**4 * bin_means))


def count_residues(u):
    count_dict = {}
    for residue in u.residues:
        if residue.resname not in count_dict:
            count_dict[residue.resname] = 1
        else:
            count_dict[residue.resname] += 1
    return count_dict


### Load data from pickle

In [ ]:
with open(util.analysis_path / "mc_noobject.pickle", "rb") as handle:
    mc = pickle.load(handle)


In [ ]:
## LOAD MEMBRANE CURVATURE OBJECTS
# mc = {}
# for sim in util.simulations:
#     with open(util.analysis_path / f"{sim}/membrane_curvature.pickle", "rb") as handle:
#         mc[sim] = pickle.load(handle)


### System information

In [11]:
def get_compositions(sim):
    top = util.analysis_path / sim / "system.top"
    gro = util.analysis_path / sim / "membrane_only.gro"

    u = MDAnalysis.Universe(top, gro, topology_format="ITP")
    raw_composition = count_residues(u)

    total_lipids = 0
    for lipid in util.lipid_names:
        if lipid in raw_composition:
            total_lipids += raw_composition[lipid]

    normed_composition = {}
    s = ""
    for lipid in util.lipid_names:
        if lipid in raw_composition:
            s += f"{lipid}: {raw_composition[lipid]/total_lipids:0.2f}; "
            normed_composition[lipid] = raw_composition[lipid] / total_lipids
        else:
            s += f"{lipid}: {0:0.2f}; "
            normed_composition[lipid] = 0
    print(sim, total_lipids)    
    return sim, raw_composition, normed_composition, s


result = map(get_compositions, util.simulations)

compositions = {}
for sim, raw, normed, s in result:
    print(f"System {sim}: {s}")
    compositions[sim, "raw_composition"] = raw
    compositions[sim, "normed_composition"] = normed


1 5404
System 1: CDL1: 0.12; DOPG: 0.00; POPG: 0.00; DOPE: 0.27; POPE: 0.03; DOPC: 0.46; POPC: 0.12; 
2 5406
System 2: CDL1: 0.12; DOPG: 0.00; POPG: 0.00; DOPE: 0.29; POPE: 0.08; DOPC: 0.25; POPC: 0.26; 
3 5402
System 3: CDL1: 0.12; DOPG: 0.00; POPG: 0.00; DOPE: 0.18; POPE: 0.14; DOPC: 0.22; POPC: 0.34; 
4 5404
System 4: CDL1: 0.00; DOPG: 0.05; POPG: 0.06; DOPE: 0.22; POPE: 0.02; DOPC: 0.55; POPC: 0.10; 
5 5404
System 5: CDL1: 0.00; DOPG: 0.02; POPG: 0.12; DOPE: 0.25; POPE: 0.07; DOPC: 0.34; POPC: 0.20; 
6 5402
System 6: CDL1: 0.00; DOPG: 0.02; POPG: 0.12; DOPE: 0.18; POPE: 0.14; DOPC: 0.20; POPC: 0.34; 
7 5408
System 7: CDL1: 1.00; DOPG: 0.00; POPG: 0.00; DOPE: 0.00; POPE: 0.00; DOPC: 0.00; POPC: 0.00; 
8 5406
System 8: CDL1: 0.20; DOPG: 0.00; POPG: 0.00; DOPE: 0.00; POPE: 0.30; DOPC: 0.00; POPC: 0.50; 
9 5406
System 9: CDL1: 0.20; DOPG: 0.00; POPG: 0.00; DOPE: 0.30; POPE: 0.00; DOPC: 0.50; POPC: 0.00; 
10 5406
System 10: CDL1: 0.00; DOPG: 0.20; POPG: 0.00; DOPE: 0.00; POPE: 0.30; DOP

In [9]:
print(get_compositions("3"))

ValueError: The topology and GRO trajectory files don't have the same number of atoms!
Topology number of atoms 74544
Trajectory: /Users/ctlee/Downloads/mito_lipidomics/analysis/3/production5.gro Number of atoms 340213

In [5]:
print(compositions[sim, "raw_composition"])

NameError: name 'sim' is not defined

## Defining Statistical Inefficiency

Given a sequence of measurements $A_i$ sampled from a timeseries, we must investigate the degree of correlation to estimate the sampling error. We estimate the error by quantifying the statistical inefficiency.

We start by computing the block averaged values, $\langle A\rangle_b$ over a range of block lengths $t_b$,
$$\langle A\rangle_b = \frac{1}{t_b} \sum_{i=1}^{t_b} A_i.$$

As the number of steps increases, we expect that the block averages become uncorrelated. The variance of block averages $\sigma^2(\langle A\rangle_b)$,
$$\sigma^2(\langle A\rangle_b) = \frac{1}{n_b}\sum_{b=1}^{n_b} (\langle A\rangle_b - \langle A_i\rangle)^2,$$
becomes inversely proportional to $t_b$ as the block averages become uncorrelated.

At the uncorrelated limit, the statistical inefficiency is given by,
$$ s = \lim_{t_b\rightarrow \infty} \frac{t_b \sigma^2(\langle A\rangle_b)}{\sigma^2(A)}.$$

The 'true' standard deviation of the average value is then related to the traditional standard deviation by,
$$\sigma_{\langle A\rangle} \approx \sigma \sqrt{\frac{s}{M}}.$$

## Setting up parametric error analysis

Given a sequence of evenly space measurements ${X_1, X_2,\ldots, X_T}$ along a trajectory, the sample mean $m_X$ and sample variance $s^2_X$ is given by

$$m_X = \frac{1}{T} \sum_{i=1}^{T}X_i,$$
$$s^2_X = \frac{1}{T-1}\sum_{i=1}^{T}(X_i - m_X)^2.$$

The error of the mean can be estimated using $\delta X = s_X / \sqrt{T}$ if the data are uncorrelated. Since the measurements are sampled from a dynamical trajectory, there is no guarantee that there is no correlation.


## Kc Bending Modulus

In [ ]:
kc_low_q = 0.4 / 10  # A^-1

def fit_kc_from_power(
    power2D, mc=None, threshold=0.03, min_bin=0.001, max_bin=1, bin_width=0.001
):
    spectra = radial_averaging(
        power2D, mc, min_bin=min_bin, max_bin=max_bin, bin_width=bin_width
    )
    mask = spectra[:, 0] < threshold
    spectra_cut = spectra[mask, :]

    return 1.0 / spectra_cut[:, 2].mean()


In [ ]:
areas = {}

for sim in util.simulations:
    gro = util.analysis_path / f"{sim}/po4_only.gro"
    traj = util.analysis_path / f"{sim}/po4_all.xtc"

    u = MDAnalysis.Universe(gro, str(traj), refresh_offsets=True)
    dims = [u.dimensions[0] for ts in u.trajectory]
    print(
        f"{sim}: mean {np.mean(dims)}, min {np.min(dims)}, max {np.max(dims)} Angstroms"
    )
    areas[sim] = np.mean(dims) ** 2


### Block analysis of Fourier modes

In [ ]:
# Override and recompute even if spectra pickle exists
spectra_compute_override = False

spectra_fd = util.analysis_path / "spectra.pickle"
if spectra_fd.exists() and not spectra_compute_override:
    # LOAD SPECTRA PICKLE
    with open(spectra_fd, "rb") as handle:
        spectra = pickle.load(handle)
    print("Loaded spectra from cache!")
else:
    def compute_spectra(sim):
        return sim, radial_averaging_series(
            mc[sim]["height_power_spectrum"],
            mc[sim],
            min_bin=0.001,
            max_bin=1,
            bin_width=0.001,
        )

    spectra = dict(map(compute_spectra, util.simulations))

    # WRITE SPECTRA TO PICKLE
    with open(spectra_fd, "wb") as handle:
        pickle.dump(spectra, handle, protocol=pickle.HIGHEST_PROTOCOL)


# POPULATE q^4*h_q
qfour_spectra = {}
for sim in util.simulations:
    qfour_spectra[sim] = np.power(spectra[sim][0], 4) * spectra[sim][1]

#### Timeseries of wavenumbers

In [ ]:
show_figs = False
curr_fig_path = Path("Figures/power_timeseries")
curr_fig_path.mkdir(parents=True, exist_ok=True)

## PLOT HEIGHT POWER TIMESERIES
for sim in util.simulations:
    fig, ax = plt.subplots(1, 1, figsize=(3, 3))  # sharex=True,

    for i in range(0, 4):
        ax.plot(
            range(len(spectra[sim][1][:, i])),
            spectra[sim][1][:, i],
            linewidth=NORMAL_LINE,
            label=f"q{i}",
        )
        
        # plot q^4*h_q instead
        # ax.plot(
        #     range(len(spectra[sim][1][:, i])),
        #     np.power(spectra[sim][0][i],4)*spectra[sim][1][:, i],
        #     linewidth=NORMAL_LINE,
        #     label=f"q{i}",
        # )

    ax.set_xlabel(r"Frame")
    ax.set_ylabel(r"Power Spectrum $\langle|h_q|\rangle^2$ (nm$^{-4}$)")
    ax.set_title(f"Sys{sim}:{util.system_names[sim]}")

    ax.legend(loc="upper right")

    # # Shrink current axis by 20%
    # box = ax.get_position()
    # ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

    # # Put a legend to the right of the current axis
    # ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))

    fig.tight_layout()
    fig.savefig(curr_fig_path/f"{sim}_power_timeseries.png", format="png")

    if show_figs:
        plt.show()
        
    fig.clear()
    plt.close(fig)


#### Statistical inefficiency of Fourier amplitudes

In [ ]:
show_figs = False
curr_fig_path = Path("Figures/amplitude_si")
curr_fig_path.mkdir(parents=True, exist_ok=True)

## COMPUTE STATISTICAL INEFFICIENCY OF WAVE NUMBERS UP TO max_q
max_q = 4
discards = np.arange(0, 60, 10)
blocks = np.arange(1, 2**8 + 1, 1)

cmap = mpl.cm.get_cmap("viridis")

for sim in util.simulations:
    for q in range(0, max_q):
        fig, ax = plt.subplots(1, 1, figsize=(3, 3))  # sharex=True,
        c = cmap(np.linspace(0, 1, len(discards)))

        _, _, si = util.statistical_inefficiency(
            qfour_spectra[sim][:, q], blocks, discards
        )

        for d, discard in enumerate(discards):
            ax.plot(
                blocks,
                si[d],
                color=c[d],
                linewidth=NORMAL_LINE,
                label=f"{100-discard}%",
            )

        ax.set_xlabel(r"Blocks")
        ax.set_ylabel(r"Statistical inefficiency")
        ax.set_title(f"Sys {sim}: {util.system_names[sim]}, q4Hq{q}")
        ax.legend(loc="upper left")
        fig.tight_layout()

        # # Shrink current axis by 20%
        # box = ax.get_position()
        # ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

        # # Put a legend to the right of the current axis
        # ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))

        fig.savefig(curr_fig_path/f"{sim}_q4Hq{q}.png", format="png")

        if show_figs:
            plt.show()
        fig.clear()
        plt.close(fig)

Conclude that the wavenumbers appear to equilibrate rapidly and we can keep the majority of the trajectory. Henceforth we will discard 10% from the beginning.

#### Block averaging of amplitudes

The correlation time of the squared standard error of the mean should follow such a trend:
$$\frac{\delta X^2_b}{\delta X^2_1} = \frac{1+c_t}{1-c_t} - \frac{2*c_t}{b} * \frac{1-c^b_t}{(1-c_t)^2}$$

In [ ]:
def correlation_time_sqrt(b, tau):
    ct = np.exp(-1 / tau)
    cb = np.exp(-b / tau)
    return np.sqrt((1 + ct) / (1 - ct) - (2 * ct) / b * (1 - cb) / np.power(1 - ct, 2))

def correlation_time(b, tau):
    ct = np.exp(-1 / tau)
    cb = np.exp(-b / tau)
    return (1 + ct) / (1 - ct) - (2 * ct) / b * (1 - cb) / np.power(1 - ct, 2)

In [ ]:
# Discard first X% for all trajectories
discard = 10
max_q_dict = {}
blocks = np.arange(1, 2**9 + 1, 1)

block_var = {}
lp_block_sem = {}
block_mean = {}
for sim in util.simulations:
    max_q = sum(spectra[sim][0] < kc_low_q)

    max_q_dict[sim]  = max_q

    block_mean[sim] = np.zeros((max_q, len(blocks)))
    block_var[sim] = np.zeros((max_q, len(blocks)))
    lp_block_sem[sim] = np.zeros((max_q, len(blocks)))

    low_q_data = qfour_spectra[sim][:, 0:max_q]
    # low_q_data = spectra[sim][1][:, 0:max_q]
    
    _, remainder = np.split(low_q_data, [int(discard / 100 * len(low_q_data))])

    block_mean[sim] = util.nd_block_average(remainder, axis=0, func=np.mean, blocks=blocks)
    block_var[sim] = util.nd_block_average(remainder, axis=0, func=partial(np.var, ddof=1), blocks=blocks)
    lp_block_sem[sim] = util.nd_block_average(remainder, axis=0, func=partial(stats.sem, ddof=1), blocks=blocks)


In [ ]:
show_figs = False
curr_fig_path = Path("Figures/kc_block_error")
curr_fig_path.mkdir(parents=True, exist_ok=True)

corrected_mean_sem = {}

for sim in util.simulations:
    fig, ax = plt.subplots(1, 1, figsize=(3, 3))  # sharex=True,

    corrected_mean_sem[sim] = np.empty((2, max_q_dict[sim]))

    for q in range(0, max_q_dict[sim]):
        # Mean with block size 1
        corrected_mean_sem[sim][0, q] = block_mean[sim][q][0] 
        blocked_sem = lp_block_sem[sim][q]
        popt, pcov = curve_fit(
            correlation_time_sqrt, blocks, blocked_sem / blocked_sem[0]
        )
        corrected_mean_sem[sim][1, q] = blocked_sem[0]* np.sqrt(2*popt[0])
        
        ax.plot(
            np.log2(blocks),
            blocked_sem / blocked_sem[0],
            linewidth=NORMAL_LINE,
            label=f"q{q}",
            color=sns.color_palette("colorblind")[q],
            linestyle=":",
        )

        ax.plot(
            np.log2(blocks),
            [correlation_time_sqrt(block, popt) for block in blocks],
            linewidth=NORMAL_LINE,
            color=sns.color_palette("colorblind")[q],
        )

    ax.set_xlabel(r"$log_2$(block)")
    ax.set_ylabel(r"$\delta X_b/\delta X_1$")
    ax.set_title(f"Sys{sim}:{util.system_names[sim]}")
    ax.legend(loc="upper left")

    fig.tight_layout()

    fig.savefig(curr_fig_path/f"{sim}_block_error.png", format="png")

    if show_figs:
        plt.show()
    fig.clear()
    plt.close(fig)


In [ ]:

# for i in range(4):
#     print(f"q^4h_q ({i}) = {corrected_mean_sem["4"][0]} +- {corrected_mean_sem["4"][1]}")

In [ ]:
# # Discard first X% for all trajectories
# discard = 10
# max_q = 4
# blocks = np.arange(1, 2**9 + 1, 1)

# for sim in util.simulations:
#     low_q_data = qfour_spectra[sim][:, 0:max_q]
#     # low_q_data = spectra[sim][1][:, 0:max_q]
    
#     _, remainder = np.split(low_q_data, [int(discard / 100 * len(low_q_data))])

#     hq_mean = np.mean(remainder, axis=0)

#     util.nd_block_average(remainder, axis=0, func=np.mean, blocks=blocks)
#     block_sem = util.nd_block_average(remainder, axis=0, func=partial(stats.sem, ddof=1), blocks=blocks)

#     corrected_hq_sem = np.empty((max_q))

#     for q in range(max_q):
#         blocked_sem = block_sem[q]
#         popt, pcov = curve_fit(
#             correlation_time_sqrt, blocks, block_sem[q] / block_sem[q][0]
#         )
#         corrected_hq_sem[q] = block_sem[q][0]* np.sqrt(2*popt[0])
    

In [ ]:
show_figs = True
curr_fig_path = Path("Figures")
curr_fig_path.mkdir(parents=True, exist_ok=True)


cmap = mpl.cm.get_cmap("viridis")
c = cmap(np.linspace(0, 1, len(util.simulations)))

kc_mean_std = np.empty((len(util.simulations), 2))

# Bootstrap values
fig, ax = plt.subplots(1, 1, figsize=(5, 4))
for i, sim in enumerate(util.simulations):
    f_rvs = []

    # Construct random Gaussian processe for each wavenumber
    for q in range(corrected_mean_sem[sim].shape[1]):
        # print(corrected_mean_sem[sim][0, i], corrected_mean_sem[sim][1][i])
        r = stats.norm(
            loc=corrected_mean_sem[sim][0, q], scale=corrected_mean_sem[sim][1][q]
        )
        f_rvs.append(r.rvs)
    # Run parametric bootstrap with random proceses
    boot = util.parametric_bootstrap(f_rvs, n_samples=50000)
    
    # Fit kcs to bootstrap samples
    kcs = 1.0 / np.mean(boot, axis=0)

    # Plot distribution of kcs
    ax.hist(
        kcs,
        bins=50,
        density=True,
        color=c[i],
        label=f"{sim} {util.system_names[sim]}",
    )

    kc_mean_std[i] = [np.mean(kcs), np.std(kcs)]

    print(
        sim, kc_mean_std[i]
    )

ax.set_xlabel(r"$K_c$ ($k_BT$)")
ax.set_ylabel("Density")

# Shrink current axis by 20%
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

# Put a legend to the right of the current axis
ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))

fig.savefig(curr_fig_path / f"kc_distributions.png", format="png")

if show_figs:
    plt.show()
fig.clear()
plt.close(fig)


In [ ]:
show_figs = False
curr_fig_path = Path("Figures/height_spectra_kc")
curr_fig_path.mkdir(parents=True, exist_ok=True)

# # Discard first X% for all trajectories
discard = 10
max_q = 100

for i, sim in enumerate(util.simulations):
    fig, ax = plt.subplots(1, 1, figsize=(3, 3))

    low_q_data = qfour_spectra[sim][:, 0:max_q]
    # low_q_data = spectra[sim][1][:, 0:max_q]

    _, remainder = np.split(low_q_data, [int(discard / 100 * len(low_q_data))])

    # Convert to nm^-1
    q = spectra[sim][0][0:max_q] * 10
    mask = q < kc_low_q * 10

    # q^4*hq
    hq_mean = np.mean(remainder, axis=0)
    hq_std = np.std(remainder, axis=0)


    ax.errorbar(q[mask], hq_mean[mask], yerr=hq_std[mask], fmt=".", markersize=3, elinewidth=THIN_LINE, ecolor="lightgray")

    ax.errorbar(q[~mask], hq_mean[~mask], yerr=hq_std[~mask], color="dimgray", fmt=".", markersize=3, elinewidth=THIN_LINE, ecolor="lightgray")

    ax.axhline(1/kc_mean_std[i][0], color="r")
    # ax.axvline(kc_low_q, color="k", linewidth=0.5, linestyle=":")

    ax.text(
        0.05,
        0.7,
        f"$K_c$ = {kc_mean_std[i][0]:.1f} $\pm$ {kc_mean_std[i][1]:.1f} $k_BT$",
        color="r",
        transform=ax.transAxes
    )

    # ax.set_xlim(5e-2, 5)
    ax.set_ylim(0.0, 0.45)
    ax.set_xscale("log")

    ax.set_ylabel(r"$q^4 \times \mathrm{intensity}$ (nm$^2$)")
    ax.set_xlabel(r"$q$ (nm$^{-1}$)")

    ax.set_title(f"System {sim}: {util.system_names[sim]}")
    fig.set_tight_layout(True)


    fig.savefig(curr_fig_path/f"{sim}_height_spectra_kc.png", format="png")

    if show_figs:
        plt.show()
        
    fig.clear()
    plt.close(fig)

In [ ]:
show_figs = True
curr_fig_path = Path("Figures")
curr_fig_path.mkdir(parents=True, exist_ok=True)

c1 = mpl.cm.get_cmap("Purples")(np.linspace(0.2, 0.8, 3))
c2 = mpl.cm.get_cmap("Reds")(np.linspace(0.2, 0.8, 3))

c3 = np.array([0.2, 0.2, 0.2, 1]).reshape(1, -1)

c4 = mpl.cm.get_cmap("Blues")(np.linspace(0.8, 0.2, 2))
c5 = mpl.cm.get_cmap("Oranges")(np.linspace(0.8, 0.2, 2))

colors = np.vstack((c1, c2, c3, c4, c5))

fig, ax = plt.subplots(1, 1, figsize=(3, 3))

ax.bar(
    range(kc_mean_std.shape[0]), kc_mean_std[:, 0], color=colors, yerr=kc_mean_std[:, 1]
)

ax.set_ylabel(r"$K_c$ ($k_BT$)")
ax.set_xlabel(r"System")

ax.set_xticklabels(
    ax.get_xticks(),
)

x_ticks_labels = [f"{sim}" for sim in util.simulations]

# Set number of ticks for x-axis
ax.set_xticks(range(11))
# Set ticks labels for x-axis
ax.set_xticklabels(x_ticks_labels)

fig.tight_layout()

fig.savefig(curr_fig_path / f"estimated_kcs.png", format="png")

if show_figs:
    plt.show()

fig.clear()
plt.close(fig)


### Statistical Inefficiency

In [ ]:
# UNCOMMENT TO REGENERATE DATA

# _s = 10  # Min block

# discards = np.arange(0, 100, 10)  # Percent of data to discard from end
# blocks = np.arange(1, 101, 1, dtype=int)

# for sim in util.simulations:
#     print("Processing system:", sim)

#     # Populate some basic information
#     frame_dt = mc[sim].times[1] - mc[sim].times[0]  # picoseconds

#     print(frame_dt)
#     _b = 0
#     _e = len(mc[sim].frames)
#     n_frames = len(range(_b, _e + 1, _s))
#     shape = mc[sim].P.shape

#     # Times from mdanalysis are unreliable due to restarts
#     # times = mc[sys].times[np.arange(0, _e, _s, dtype=int)]
#     times = np.arange(0, _e, _s, dtype=int) * frame_dt  # picoseconds

#     # Split into chunks and compute kcs
#     split_indices = np.arange(_s, _e, _s, dtype=int)
#     powers = np.fromiter(
#         map(
#             partial(np.mean, axis=0),
#             np.split(mc[sim].results["height_power_spectrum"], split_indices),
#         ),
#         dtype=np.dtype((np.double, (shape))),
#     )
#     kcs = np.fromiter(
#         map(partial(fit_kc_from_power, threshold=kc_low_q, mc=mc[sim]), powers),
#         dtype=np.double,
#     )

#     # Plot Kc evaluated over 5ns (0.5 * _s) blocks
#     fig, ax = plt.subplots(1, 1, figsize=(3, 3))  # sharex=True,
#     # fig.subplots_adjust(hspace=0.05)  # adjust space between axes

#     ax.plot(times * 1e-6, kcs)  # Convert to microseconds
#     # ax2.plot(times * 1e-6, kcs)  # Convert to microseconds

#     # ax1.set_ylim(50, 100)
#     # ax2.set_ylim(0, 30)

#     # # hide the spines between ax and ax2
#     # ax1.spines.bottom.set_visible(False)
#     # ax2.spines.top.set_visible(False)
#     # ax1.xaxis.tick_top()
#     # ax1.tick_params(labeltop=False)  # don't put tick labels at the top
#     # ax2.xaxis.tick_bottom()

#     # # Now, let's turn towards the cut-out slanted lines.
#     # # We create line objects in axes coordinates, in which (0,0), (0,1),
#     # # (1,0), and (1,1) are the four corners of the axes.
#     # # The slanted lines themselves are markers at those locations, such that the
#     # # lines keep their angle and position, independent of the axes size or scale
#     # # Finally, we need to disable clipping.

#     # d = .5  # proportion of vertical to horizontal extent of the slanted line
#     # kwargs = dict(marker=[(-1, -d), (1, d)], markersize=12,
#     #             linestyle="none", color='k', mec='k', mew=1, clip_on=False)
#     # ax1.plot([0, 1], [0, 0], transform=ax1.transAxes, **kwargs)
#     # ax2.plot([0, 1], [1, 1], transform=ax2.transAxes, **kwargs)

#     ax.set_ylabel(r"$K_c$ ($k_b$T)")
#     ax.set_xlabel(r"Time (μs)")

#     ax.set_title(f"System {sim}: {util.system_names[sim]}")

#     # fig.tight_layout()
#     save_fig(fig, f"Figures/{sim}_kc_5nsblock")

#     ## Perform block analysis
#     kc_mean = kcs.mean()

#     SI = {}
#     SI["discards"] = discards
#     SI["blocks"] = blocks * frame_dt * _s  # picoseconds

#     for discard in discards:
#         _, r = np.split(kcs, [int(discard / 100 * n_frames)])
#         kc_mean = r.mean()

#         SI[(discard, "sigma")] = np.zeros(len(blocks))
#         SI[(discard, "mean")] = np.zeros(len(blocks))

#         for i, block in enumerate(blocks):
#             n = np.floor(len(r) / block)

#             split_indices = np.arange(block, len(r), block, dtype=int)
#             block_kcs = np.fromiter(
#                 map(np.mean, np.split(r, split_indices)), dtype=np.double
#             )

#             # Truncate number of blocks if not evenly divisible
#             if len(r) % block:
#                 block_kcs = block_kcs[:-1]

#             SI[(discard, "sigma")][i] = np.sum((block_kcs - kc_mean) ** 2) / n
#             SI[(discard, "mean")][i] = np.mean(block_kcs)

#     with open(util.analysis_path / f"{sim}/SI.pickle", "wb") as handle:
#         pickle.dump(SI, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
# # Load bending modulus statistical inefficiency
# SI_kc = {}
# for sim in util.simulations:
#     with open(util.analysis_path / sim / "SI.pickle", "rb") as handle:
#         SI_kc[sim] = pickle.load(handle)


In [ ]:
# # Foreshadow percent discard, statistical inefficiency for all systems
# si_kc_system = {
#     "1": (50, 25),
#     "2": (50, 25),
#     "3": (50, 25),
#     "4": (50, 13),
#     "5": (20, 15),
#     "6": (20, 15),
#     "7": (50, 28),
#     "8": (50, 25),
#     "9": (50, 25),
#     "10": (50, 25),
#     "11": (50, 25),
# }

# for sim in util.simulations:
#     print(f"System {sim} ({util.system_names[sim]}):")
#     print(f"  statistical inefficiency: {si_kc_system[sim][1]} ns")
#     print(f"                   discard: {si_kc_system[sim][0]}%")


In [ ]:
# cmap = mpl.cm.get_cmap("viridis")

# for sim in util.simulations:
#     d = SI_kc[sim]
#     discards = d["discards"]
#     c = cmap(np.linspace(0, 1, len(discards)))

#     times = d["blocks"] * 1e-3  # picoseconds -> nanoseconds

#     fig, ax = plt.subplots(1, 1, figsize=(4, 3))

#     for i, discard in enumerate(discards):
#         sigma = d[(discard, "sigma")]
#         si = times * (sigma / sigma[0])

#         if discard == si_kc_system[sim][0]:
#             alpha = 1
#         else:
#             alpha = 0.2
#         ax.plot(
#             times,
#             si,
#             color=c[i],
#             alpha=alpha,
#             linewidth=NORMAL_LINE,
#             label=f"{100-discard}%",
#         )

#     ax.axvline(100, color="k", linestyle="dotted", linewidth=THINNER_LINE)
#     ax.axhline(
#         si_kc_system[sim][1], color="k", linestyle="dotted", linewidth=THINNER_LINE
#     )

#     ax.set_ylabel(r"Statistical Inefficiency (ns)")
#     ax.set_xlabel(r"Bin Width (ns)")
#     ax.set_title(f"System {sim}: {util.system_names[sim]}")
#     # ax.legend()

#     # Shrink current axis by 20%
#     box = ax.get_position()
#     ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

#     # Put a legend to the right of the current axis
#     ax.legend(loc="center left", bbox_to_anchor=(1, 0.5), title="Keep")

#     ax.set_xlim(0, 200)
#     ax.set_ylim(0.0, 50)
#     # plt.tight_layout()
#     save_fig(fig, f"Figures/{sim}_StatIneff")
#     plt.show()
#     fig.clear()
#     plt.close(fig)


### Plots of height power spectrum

In [ ]:
# _s = 10  # Min block
# cmap = mpl.cm.get_cmap("viridis")

# fig_all, ax_all = plt.subplots(1, 1, figsize=(6, 3))
# KB_VALUES = {}
# for sim in util.simulations:
#     print("Processing system:", sim)
#     discard = si_kc_system[sim][0]
#     si = si_kc_system[sim][1] * 1e3 / 500  # nanseconds -> picoseconds -> frames

#     _, r = np.split(
#         mc[sim].results.height_power_spectrum,
#         [int(discard / 100 * len(mc[sim].results.height_power_spectrum))],
#     )

#     m = SI_kc[sim][(discard, "mean")][0]
#     sigma = SI_kc[sim][(discard, "sigma")][0]

#     KB_VALUES[sim] = m

#     N = len(r)
#     stdev = sigma * np.sqrt(si / N)

#     print(m, sigma, N, si, stdev)

#     c = cmap(np.linspace(0, 1, len(util.simulations)))

#     height_spectra = radial_averaging(np.mean(r, axis=0), mc[sim])

#     mask = height_spectra[:, 0] < kc_low_q
#     height_spectra[:, 0] *= 10  # Convert to nm^-1
#     height_spectra_cut = height_spectra[mask, :]
#     height_spectra_r = height_spectra[~mask, :]

#     fig, ax = plt.subplots(1, 1, figsize=(3, 3))

#     ax.scatter(height_spectra_r[:, 0], height_spectra_r[:, 2], color="silver", s=8)
#     ax.scatter(height_spectra_cut[:, 0], height_spectra_cut[:, 2], s=8)

#     ax.axhline(1 / m, color="r")
#     ax.axvline(kc_low_q, color="k", linewidth=0.5, linestyle=":")

#     ax.set_xlim(5e-2, 2.5)
#     ax.set_ylim(0.0, 0.4)
#     ax.set_xscale("log")

#     ax.text(
#         0.1,
#         0.2,
#         f"$K_c$ = {m:.1f} $\pm$ {stdev:.1f} $k_BT$",
#         color="r",
#     )

#     ax.set_ylabel(r"$q^4 \times \mathrm{intensity}$ (-)")
#     ax.set_xlabel(r"$q$ (nm$^{-1}$)")
#     ax.set_title(f"System {sim}: {util.system_names[sim]}")
#     fig.set_tight_layout(True)

#     save_fig(fig, f"Figures/{sim}_kc")
#     # plt.show()
#     # fig.clear()
#     # plt.close(fig)

#     ax_all.scatter(
#         height_spectra[:, 0],
#         height_spectra[:, 2],
#         color=c[int(sim) - 1],
#         label=f"System {sim} {util.system_names[sim]}",
#     )

# # Shrink current axis by 20%
# box = ax_all.get_position()
# ax_all.set_position([box.x0, box.y0, box.width * 0.8, box.height])

# # Put a legend to the right of the current axis
# ax_all.legend(loc="center left", bbox_to_anchor=(1, 0.5))

# ax_all.set_xlim(5e-2, 2.5)
# ax_all.set_ylim(0.0, 0.4)
# ax_all.set_xscale("log")
# ax_all.set_ylabel(r"$q^4 \times \mathrm{intensity}$ (-)")
# ax_all.set_xlabel(r"$q$ (nm$^{-1}$)")
# fig_all.set_tight_layout(True)
# save_fig(fig_all, "height_spectrum-all")


In [ ]:

# temp_kc = np.empty((len(util.simulations), 2))
# for i, sim in enumerate(util.simulations):
#     discard = si_kc_system[sim][0]
#     si = si_kc_system[sim][1] * 1e3 / 500  # nanseconds -> picoseconds -> frames
#     _, r = np.split(
#         mc[sim].results.height_power_spectrum,
#         [int(discard / 100 * len(mc[sim].results.height_power_spectrum))],
#     )
    
#     m = SI_kc[sim][(discard, "mean")][0]
#     sigma = SI_kc[sim][(discard, "sigma")][0]

#     N = len(r)
#     stdev = sigma * np.sqrt(si / N)

#     temp_kc[i] = [m, stdev]
    
# np.savetxt("sheets/kb.csv", temp_kc, delimiter=",")

     

# Neighbor analysis

In [ ]:
pd.options.display.float_format = "{:.3f}".format

# Lipids in order in results matrix
lipids = ["POPC", "DOPC", "POPE", "DOPE", "CDL1", "POPG", "DOPG"]
lipid_dict = dict([[j, i] for i, j in enumerate(lipids)])
leaflets = ["upper", "lower"]


def check_symmetric(a, rtol=1e-05, atol=1e-08):
    return np.allclose(a, a.T, rtol=rtol, atol=atol)


def print_neighbor_analysis(dataset, prefix=""):
    for sim in util.simulations:
        print(f"System {sim}: {util.system_names[sim]}")

        raw_baseline = []
        baseline = []
        s = ""
        for lipid in lipids:
            if lipid in compositions[sim, "raw_composition"]:
                # div by 2 to account for leaflet composition
                raw_baseline.append(compositions[sim, "raw_composition"][lipid] / 2)
            else:
                raw_baseline.append(0)
            s += f"{lipid}: {compositions[sim, 'normed_composition'][lipid]:0.3f}, "
            baseline.append(compositions[sim, "normed_composition"][lipid])
        raw_baseline = np.array(raw_baseline).reshape(-1, 1)  # COLUMN VECTOR
        print(s)
        # fraction of each lipid per leaflet
        baseline = np.array(baseline)
        # print(raw_baseline, baseline)

        # Aggregate statistics from both leaflets
        counts_per_frame = (dataset[sim]["upper"] + dataset[sim]["lower"]) / 2
        # Keep only back half of trajectory
        N = int(0.5 * len(counts_per_frame))
        counts = np.mean(counts_per_frame[N:-1], axis=0)  # mean counts per frame

        # Copy counts across diagonal which must be symmetric
        for i in range(0, len(lipids)):
            for j in range(0, i):
                counts[i, j] = counts[j, i]
            counts[i, i] *= 2
        counts /= 2

        # print(pd.DataFrame(counts, columns=lipids, index=lipids))

        # Normalize by number of each lipid within leaflet
        # ROWWISE DIVISION...
        normed_counts = np.divide(
            counts, raw_baseline, out=np.zeros_like(counts), where=raw_baseline != 0
        )
        df_normed_counts = pd.DataFrame(normed_counts, columns=lipids, index=lipids)
        print("# of each lipid around row lipid\n", df_normed_counts)

        rowwise_sum = np.sum(normed_counts, axis=1).reshape(-1, 1)
        # print(np.array2string(rowwise_sum, max_line_width=np.inf))

        likelihood_count = np.divide(
            normed_counts,
            rowwise_sum,
            out=np.zeros_like(counts),
            where=rowwise_sum != 0,
        )

        # print(np.array2string(likelihood_count, max_line_width=np.inf))

        diff_counts = np.divide(
            likelihood_count,
            baseline,
            out=np.zeros_like(normed_counts),
            where=baseline != 0,
        )
        # df = pd.DataFrame(normed_counts, columns=lipids, index=lipids)
        # print(df)

        # print("DIFF:")
        df_diff = pd.DataFrame(pd.DataFrame(diff_counts, columns=lipids, index=lipids))

        df_diff.to_csv(f"sheets/sim_{sim}_likelihood_{prefix}.csv")

        print("Enrichment from likelihood:\n", df_diff)
        print()


In [ ]:
neighbor_enrichment = {}

for sim in util.simulations:
    with open(
        util.analysis_path / f"{sim}/neighbor_enrichment_leaflet_glo.pickle", "rb"
    ) as handle:
        neighbor_enrichment[sim] = pickle.load(handle)


In [ ]:
print_neighbor_analysis(neighbor_enrichment, prefix="15A")

In [ ]:
voronoi = {}

for sim in util.simulations:
    with open(util.analysis_path / f"{sim}/voronoi_leaflet_glo.pickle", "rb") as handle:
        voronoi[sim] = pickle.load(handle)

In [ ]:
print_neighbor_analysis(voronoi, prefix="voronoi")

# Lateral Pressures

In [ ]:
def first_cubic_np(z, lp, minz=None, maxz=None, maxiter=5000, sym: bool = True):
    """
    Compute the first moment using a cubic piecewise interpolant and Gaussian quadrature
    """
    lp = lp / 1e3  # to convert to piconewtons / nm^2

    if sym:
        if len(lp) % 2 == 1:
            s = int(np.floor(len(lp) / 2))
            bot, mid, top = np.split(lp, [s, s + 1])
            sym_top = (np.flip(bot) + top) / 2
            sym_bot = np.flip(sym_top)
            lp = np.hstack((sym_bot, mid, sym_top))
        else:
            bot, top = np.split(lp, 2)
            sym_top = (np.flip(bot) + top) / 2
            sym_bot = np.flip(sym_top)
            lp = np.hstack((sym_bot, sym_top))

    y = lp * z

    if minz is None:
        minz = min(z)
    if maxz is None:
        maxz = max(z)

    approx = interpolate.interp1d(z, y, kind="cubic")  # interpolating a function
    intg = integrate.quadrature(
        approx, minz, maxz, maxiter=maxiter
    )  # finding the integral over bounds
    return intg


def zero_cubic_np(z, lp, minz=None, maxz=None, maxiter=5000, sym: bool = True):
    """
    Compute the zeroeth moment using a cubic piecewise interpolant and Gaussian quadrature
    """
    # p = data['LP_(kPA)']/1e15 # to convert to newtons / nm^2
    lp = lp / 1e3  # to convert to piconewtons / nm^2

    if sym:
        if len(lp) % 2 == 1:
            s = int(np.floor(len(lp) / 2))
            bot, mid, top = np.split(lp, [s, s + 1])
            sym_top = (np.flip(bot) + top) / 2
            sym_bot = np.flip(sym_top)
            lp = np.hstack((sym_bot, mid, sym_top))
        else:
            bot, top = np.split(lp, 2)
            sym_top = (np.flip(bot) + top) / 2
            sym_bot = np.flip(sym_top)
            lp = np.hstack((sym_bot, sym_top))

    if minz is None:
        minz = min(z)
    if maxz is None:
        maxz = max(z)

    approx = interpolate.interp1d(z, lp, kind="cubic")  # interpolating a function
    intg = integrate.quadrature(
        approx, minz, maxz, maxiter=maxiter
    )  # finding the integral over bounds
    return intg


def mean_squared_deviation_np(data: npt.ArrayLike) -> float:
    """
    Compute the mean squared deviation of the data
    """
    return sum(data ** 2) / (data.size - 1)


def max_difference(a: npt.ArrayLike, b: npt.ArrayLike):
    """
    Get the max difference between two equal sized arrays
    """
    if a.size != b.size:
        raise RuntimeError("a and b must be same sized arrays")
    diff = np.abs(a - b)
    return np.amax(diff)

In [ ]:
# def first_cubic(data, minz=None, maxz=None, maxiter=5000, sym: bool = True):
#     """
#     Compute the first moment using a cubic piecewise interpolant and Gaussian quadrature
#     """
#     x = data["z"]  # in nanometers
#     # p = data['LP_(kPA)']/1e15 #to convert to newtons / nm^2
#     p = data["LP_(kPA)"].values / 1e3  # to convert to piconewtons / nm^2

#     if sym:
#         if len(p) % 2 == 1:
#             s = int(np.floor(len(p) / 2))
#             bot, mid, top = np.split(p, [s, s + 1])
#             sym_top = (np.flip(bot) + top) / 2
#             sym_bot = np.flip(sym_top)
#             p = np.hstack((sym_bot, mid, sym_top))
#         else:
#             bot, top = np.split(p, 2)
#             sym_top = (np.flip(bot) + top) / 2
#             sym_bot = np.flip(sym_top)
#             p = np.hstack((sym_bot, sym_top))

#     y = p * x

#     if minz is None:
#         minz = min(x)
#     if maxz is None:
#         maxz = max(x)

#     approx = interpolate.interp1d(x, y, kind="cubic")  # interpolating a function
#     intg = integrate.quadrature(
#         approx, minz, maxz, maxiter=maxiter
#     )  # finding the integral over bounds
#     return intg


# def zero_cubic(data, minz=None, maxz=None, maxiter=5000, sym: bool = True):
#     """
#     Compute the zeroeth moment using a cubic piecewise interpolant and Gaussian quadrature
#     """
#     x = data["z"]  # in nanometers
#     # p = data['LP_(kPA)']/1e15 # to convert to newtons / nm^2
#     p = data["LP_(kPA)"].values / 1e3  # to convert to piconewtons / nm^2

#     if sym:
#         if len(p) % 2 == 1:
#             s = int(np.floor(len(p) / 2))
#             bot, mid, top = np.split(p, [s, s + 1])
#             sym_top = (np.flip(bot) + top) / 2
#             sym_bot = np.flip(sym_top)
#             p = np.hstack((sym_bot, mid, sym_top))
#         else:
#             bot, top = np.split(p, 2)
#             sym_top = (np.flip(bot) + top) / 2
#             sym_bot = np.flip(sym_top)
#             p = np.hstack((sym_bot, sym_top))

#     if minz is None:
#         minz = min(x)
#     if maxz is None:
#         maxz = max(x)

#     approx = interpolate.interp1d(x, p, kind="cubic")  # interpolating a function
#     intg = integrate.quadrature(
#         approx, minz, maxz, maxiter=maxiter
#     )  # finding the integral over bounds
#     return intg

# def mean_squared_deviation(data: pd.DataFrame, var: str = "Szz") -> float:
#     """
#     Compute the mean squared deviation of the data
#     """
#     return sum(data[var] ** 2) / (data[var].size - 1)

In [ ]:
from LStensor import LStensor

lateral_pressure = {}

for sim in util.simulations:
    fd = Path(util.analysis_path /  f"{sim}_small/stress_calc/frames/frame{i}.dat0")
    field = LStensor(2)
    field.g_loaddata(files=[fd], bAvg="avg")

    # stress_tensor = np.empty((20000, field.nz, 9))
    lateral_pressure[sim] = np.empty((20000, field.nz, 3))

    # 0-20000 frames in each trajectory
    for i, j in enumerate(range(1, 20001)):
        fd = Path(util.analysis_path / f"{sim}_small/stress_calc/frames/frame{j}.dat0")
        field = LStensor(2)
        field.g_loaddata(files=[fd], bAvg="avg")
        stress_tensor = field.data_grid * 100   # Convert to kPa from 10^5 Pa
        # Sxx Sxy Sxz Syx Syy Syz Szx Szy Szz
        # 0               4               8

        pXY = -0.5*(stress_tensor[:,0] + stress_tensor[:,4]).reshape(-1,1)
        pN = (-stress_tensor[:,8]).reshape(-1,1)
        lp = pXY - pN
        z = (np.arange(field.nz) * field.dz - (field.nz - 1) * field.dz / 2).reshape(-1,1)
        lateral_pressure[sim][i] = np.hstack((pN, lp, z))

In [ ]:
# Discard first X% for all trajectories
discard = 10
blocks = np.arange(1, 2**8 + 1, 1)

lp_block_var = {}
lp_block_sem = {}
lp_block_mean = {}
for sim in util.simulations:
    data = lateral_pressure[sim][:,:, 1]
    nT, nZs= data.shape

    lp_block_mean[sim] = np.zeros((nZs, len(blocks)))
    lp_block_var[sim] = np.zeros((nZs, len(blocks)))
    lp_block_sem[sim] = np.zeros((nZs, len(blocks)))
    
    _, remainder = np.split(data, [int(discard / 100 * len(data))])

    lp_block_mean[sim] = util.nd_block_average(remainder, axis=0, func=np.mean, blocks=blocks)
    lp_block_var[sim] = util.nd_block_average(remainder, axis=0, func=partial(np.var, ddof=1), blocks=blocks)
    lp_block_sem[sim] = util.nd_block_average(remainder, axis=0, func=partial(stats.sem, ddof=1), blocks=blocks)


In [ ]:
data = lateral_pressure["1"][:, :, 1][:, 105]
print(len(data))
result = np.correlate(data, data, mode='full')
plt.plot(result[int(result.size/2):])

In [ ]:
for sim in util.simulations:
    fig, ax = plt.subplots(1, 1, figsize=(3, 3))  # sharex=True,
    ax.errorbar(np.mean(lateral_pressure[sim], axis=0)[:, 2], lp_block_mean[sim][:,0], yerr=np.sqrt(lp_block_var[sim][:,0]), color="b", linewidth=THIN_LINE,ecolor="lightgray")

In [ ]:

for sim in util.simulations:
    # print(sim, lateral_pressure[sim][:,:,1].shape)
    fig, ax = plt.subplots(1, 1, figsize=(3, 3))  # sharex=True,

    for i in range(50, 150):
        data = lateral_pressure[sim][:,:,1]

        ax.plot(
            range(len(data[:, i])),
            data[:, i], 
            linewidth=NORMAL_LINE,
        )
    
    ax.set_xlabel(r"Frame")
    ax.set_ylabel(r"Pressure")
    ax.set_title(f"Sys{sim}:{util.system_names[sim]}")

    # ax.legend(loc="upper right")

    # # Shrink current axis by 20%
    # box = ax.get_position()
    # ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

    # # Put a legend to the right of the current axis
    # ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))

    fig.tight_layout()
    
    plt.show()
    fig.clear()
    plt.close(fig)

In [ ]:
show_figs = True
curr_fig_path = Path("Figures/lp_block_analysis")
curr_fig_path.mkdir(parents=True, exist_ok=True)

lp_corrected_mean_sem = {}

for sim in util.simulations:
    nZs, nBlock = lp_block_mean[sim].shape

    fig, ax = plt.subplots(1, 1, figsize=(3, 3))  # sharex=True,

    lp_corrected_mean_sem[sim] = np.empty((2, nZs))

    for z in range(100,105): #nZs):
        # Mean with block size 1
        lp_corrected_mean_sem[sim][0, z] = lp_block_mean[sim][z][0] 
        blocked_sem = lp_block_sem[sim][z]
        block_var = lp_block_var[sim][z]

        # popt, pcov = curve_fit(
        #     correlation_time_sqrt, blocks, blocked_sem / blocked_sem[0]
        # )
        # lp_corrected_mean_sem[sim][1, z] = blocked_sem[0]* np.sqrt(2*popt[0])
        
        ax.plot(
            np.log2(blocks),
            # blocked_sem / blocked_sem[0],
            blocks * (block_var / block_var[0]),
            linewidth=NORMAL_LINE,
            # label=f"q{q}",
            # color=sns.color_palette("colorblind")[q],
            linestyle=":",
        )

        # ax.plot(
        #     np.log2(blocks),
        #     [correlation_time_sqrt(block, popt) for block in blocks],
        #     linewidth=NORMAL_LINE,
        #     # color=sns.color_palette("colorblind")[q],
        # )

    ax.set_xlabel(r"$log_2$(block)")
    ax.set_ylabel(r"$\delta X_b/\delta X_1$")
    ax.set_title(f"Sys{sim}:{util.system_names[sim]}")
    # ax.legend(loc="upper left")

    fig.tight_layout()

    # fig.savefig(curr_fig_path/f"{sim}_lp_block_error.png", format="png")

    if show_figs:
        plt.show()
    fig.clear()
    plt.close(fig)


In [ ]:
show_figs = True
curr_fig_path = Path("Figures")
curr_fig_path.mkdir(parents=True, exist_ok=True)


fig, ax = plt.subplots(1, 11, figsize=(12, 8), sharex=True, sharey=True)
for sim in util.simulations:
    ax_index = int(sim) - 1
    ax[ax_index].axhline(0, linestyle="--", color="gray")

    data = np.mean(lateral_pressure[sim], axis=0)
    z = data[:, 2]

    ax[ax_index].plot(
        data[:,0] * 0.01,
        z,
        label="Normal Stress",
        linewidth=NORMAL_LINE,
        linestyle="-",
        color="k",
    )

    lateral = data[:,1] * 0.01 # bar

    # Symmetrizing
    if len(lateral) % 2 == 1:
        s = int(np.floor(len(lateral) / 2))
        bot, mid, top = np.split(lateral, [s, s + 1])
        sym_top = (np.flip(bot) + top) / 2
        sym_bot = np.flip(sym_top)
        sym_lp = np.hstack((sym_bot, mid, sym_top))
    else:
        bot, top = np.split(lateral, 2)
        sym_top = (np.flip(bot) + top) / 2
        sym_bot = np.flip(sym_top)
        sym_lp = np.hstack((sym_bot, sym_top))

    ax[ax_index].fill_betweenx(z, sym_lp, 0, label="Lateral Pressure", alpha=0.4)

    ax[ax_index].set_ylim(-4, 4)
    # ax[x,y].ticklabel_format(axis = 'x', style = 'sci', scilimits = (0,0))
    ax[ax_index].set_xlabel("S or P (bar)")
    ax[ax_index].set_title(f"{util.system_names[sim]}", rotation=45)
    if ax_index == 0:
        ax[ax_index].set_ylabel("Z (nm)")

# handles = [
#     mlines.Line2D([], [], color="black", linestyle=":", label="Normal Stress"),
#     mlines.Line2D(
#         [],
#         [],
#         color="black",
#         linestyle="-",
#         label="Lateral Stress",
#     ),
# ]

# plt.legend(handles=handles, bbox_to_anchor=(-2.5, -0.95, 0.8, 0.8), ncol=2)
plt.tight_layout()


fig.savefig(curr_fig_path / "stress.png", format="png")

if show_figs:
    plt.show()
fig.clear()
plt.close(fig)


In [ ]:
show_figs = True
curr_fig_path = Path("Figures")
curr_fig_path.mkdir(parents=True, exist_ok=True)



fig, ax = plt.subplots(1, 11, figsize=(12, 8), sharex=True, sharey=True)
for sim in util.simulations:
    ax_index = int(sim) - 1
    ax[ax_index].axhline(0, linestyle="--", color="gray")

    data = np.mean(lateral_pressure[sim], axis=0)
    z = data[:, 2]

    lateral = data[:,1] / 1e3 # pN/nm^2
    # Symmetrizing
    if len(lateral) % 2 == 1:
        s = int(np.floor(len(lateral) / 2))
        bot, mid, top = np.split(lateral, [s, s + 1])
        sym_top = (np.flip(bot) + top) / 2
        sym_bot = np.flip(sym_top)
        sym_lp = np.hstack((sym_bot, mid, sym_top))
    else:
        bot, top = np.split(lateral, 2)
        sym_top = (np.flip(bot) + top) / 2
        sym_bot = np.flip(sym_top)
        sym_lp = np.hstack((sym_bot, sym_top))

    ax[ax_index].fill_betweenx(z, z*sym_lp, 0, label="Lateral Stress", alpha=0.4)

    ax[ax_index].set_ylim(-4, 4)
    # ax[x,y].ticklabel_format(axis = 'x', style = 'sci', scilimits = (0,0))
    ax[ax_index].set_xlabel("z L_p (pN/nm)")
    ax[ax_index].set_title(f"{util.system_names[sim]}", rotation=45)
    if ax_index == 0:
        ax[ax_index].set_ylabel("Z (nm)")

# handles = [
#     mlines.Line2D([], [], color="black", linestyle=":", label="Normal Stress"),
#     mlines.Line2D(
#         [],
#         [],
#         color="black",
#         linestyle="-",
#         label="Lateral Stress",
#     ),
# ]

# plt.legend(handles=handles, bbox_to_anchor=(-2.5, -0.95, 0.8, 0.8), ncol=2)
plt.tight_layout()

fig.savefig(curr_fig_path / "first_mom_integrand.png", format="png")

if show_figs:
    plt.show()
fig.clear()
plt.close(fig)


In [ ]:
f_cubic_dat = {}
z_cubic_dat = {}
msd_dat = {}

for sim in util.simulations:
    try:
        data = np.mean(lateral_pressure[sim], axis=0)

        z = data[:, 2]
        lp = data[:, 1]
        szz = data[:, 0]

        fcd = first_cubic_np(z, lp)
        f_cubic_dat[sim] = (
            first_cubic_np(z, lp, maxz=0)[0] - first_cubic_np(z, lp, minz=0)[0]
        ) / 2
        # print(first_cubic(z, lp, minz=0)[0], first_cubic(z, lp, maxz=0)[0])

        zcd = zero_cubic_np(z, lp)
        z_cubic_dat[sim] = zero_cubic_np(z, lp)[0]

        msd = mean_squared_deviation_np(szz)
        msd_dat[sim] = msd
        print(
            f"{util.system_names[sim]}\n" f"   Zero Moment (pN/nm): {zcd}\n",
            f"     First Moment (pN): {fcd}\n",
            f"Monlayer First Moment (pN): {f_cubic_dat[sim]} {first_cubic_np(z, lp, maxz=0)[0]} {first_cubic_np(z, lp, minz=0)[0]}\n",
            f"           MSD (kPa^2): {msd}\n",
            f" Upper Leaflet Tension: {zero_cubic_np(z, lp, minz=0)}\n",
            f" Lower Leaflet Tension: {zero_cubic_np(z, lp, maxz=0)}\n",
        )
    except Exception as e:
        print(
            f"{util.system_names[sim]}\n" f"{e}\n",
        )
np.save("f_cubic_dat.npy", f_cubic_dat)
np.save("z_cubic_dat.npy", z_cubic_dat)
np.save("msd_dat.npy", msd_dat)


In [ ]:
# f_cubic_dat = {}
# z_cubic_dat = {}
# msd_dat = {}

# for sim in util.simulations:
#     file = util.analysis_path / f"{sim}_small/stress_calc/lateral_pressure.csv"

#     try:
#         data = pd.read_csv(file)

#         fcd = first_cubic(data)
#         f_cubic_dat[sim] = (
#             first_cubic(data, maxz=0)[0] - first_cubic(data, minz=0)[0]
#         ) / 2
#         # print(first_cubic(data, minz=0)[0], first_cubic(data, maxz=0)[0])

#         zcd = zero_cubic(data)
#         z_cubic_dat[sim] = zero_cubic(data)[0]

#         msd = mean_squared_deviation(data, "Szz")
#         msd_dat[sim] = msd
#         print(
#             f"{util.system_names[sim]}\n" f"   Zero Moment (pN/nm): {zcd}\n",
#             f"     First Moment (pN): {fcd}\n",
#             f"Monlayer First Moment (pN): {f_cubic_dat[sim]}\n",
#             f"           MSD (kPa^2): {msd}\n",
#             f" Upper Leaflet Tension: {zero_cubic(data, minz=0)}\n",
#             f" Lower Leaflet Tension: {zero_cubic(data, maxz=0)}\n",
#         )
#     except Exception as e:
#         print(
#             f"{util.system_names[sim]}\n" f"{e}\n",
#         )
# np.save("f_cubic_dat.npy", f_cubic_dat)
# np.save("z_cubic_dat.npy", z_cubic_dat)
# np.save("msd_dat.npy", msd_dat)


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))

ax.axhline(0, color="gray", linestyle="--")

ax.scatter(
    np.arange(0, 2),
    [z_cubic_dat[k] for k in ["8", "9"]],
    label="+CDL",
    marker="s",
    s=6**2,
)
ax.scatter(
    np.arange(0, 2),
    [z_cubic_dat[k] for k in ["10", "11"]],
    label="-CDL",
    s=8**2,
)

# ax.set_ylim(-2, 4)
ax.set_ylabel("0th mom. (pN/nm)")
# ax.tick_params(axis = 'y', labelcolor = 'darkorange')
# ax.set_xlim(0, 25)
ax.set_xticks([0, 1], labels=["PO", "DO"])
# plt.xscale('log')
ax.set_xlabel("Saturation")

plt.legend()
plt.tight_layout()
save_fig(fig, "Figures/zero_moment")

fig.clear()
plt.close(fig)


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))

# ax.axhline(0, color="gray", linestyle="--")

ax.scatter(
    np.arange(0, 2),
    [f_cubic_dat[k] for k in ["8", "9"]],
    label="+CDL",
    marker="s",
    s=6**2,
    alpha=0.5,
)
ax.scatter(
    np.arange(0, 2),
    [f_cubic_dat[k] for k in ["10", "11"]],
    label="-CDL",
    s=8**2,
    alpha=0.5,
)

# ax.set_ylim(-2, 4)
ax.set_ylabel("1st mom. (pN)")
# ax.tick_params(axis = 'y', labelcolor = 'darkorange')
# ax.set_xlim(0, 25)
ax.set_xticks([0, 1], labels=["PO", "DO"])
# plt.xscale('log')
ax.set_xlabel("Saturation")

plt.legend(loc="center")
plt.tight_layout()
save_fig(fig, "Figures/first_moment")
fig.clear()
plt.close(fig)


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))

# ax.axhline(0, color="gray", linestyle="--")

ax.scatter(
    np.arange(1, 12),
    [f_cubic_dat[k] for k in util.simulations],
    marker="s",
    s=6**2,
    alpha=0.5,
)

# ax.set_ylim(-2, 4)
ax.set_ylabel("1st mom. (pN)")
# ax.tick_params(axis = 'y', labelcolor = 'darkorange')
ax.set_xlabel("System")
ax.set_xticks(np.arange(1, 12, 1))

# plt.legend(loc='upper left')
plt.tight_layout()
save_fig(fig, "Figures/first_moment_all")
plt.show()
fig.clear()
plt.close(fig)


In [ ]:
for sim in util.simulations:
    print(f"System {sim} c0: {f_cubic_dat[sim]/(kc_mean_std[int(sim)-1][0] * 4.18336647): 0.3f} nm^-1;", 1 / (f_cubic_dat[sim]/(kc_mean_std[int(sim)-1][0] * 4.18336647)), "nm")

In [ ]:
# data = np.loadtxt("sheets/kb.csv", delimiter=",")
# print(data)


# tdata = np.empty((5, 2))
# for i, v in enumerate([10, 9, 3, 4, 5]):
#     tdata[i] = data[v]
#     print(util.system_names[str(v + 1)])
# print(tdata)
# names = [f"System {i}" for i in range(1, 6)]
# print(names)

In [ ]:
# fig, ax = plt.subplots(1, 1, figsize=(5, 5))  # sharex=True,
# ax.bar(names, tdata[:, 0], yerr=tdata[:, 1])
# ax.set_ylabel(r"Bending modulus ($k_BT$)")

# save_fig(fig, f"Figures/temp_kb")


# Extra

In [ ]:
for k, v in f_cubic_dat.items():
    print(f"{k}	{v}")

In [ ]:
for k, v in f_cubic_dat.items():
    print(f"{k}	{v}")